In [1]:
import practicalSPARQL
import pandas as pd
import requests
import os

In [2]:
root_q = 'queries'
root_d = 'results'

# read login data
login = pd.read_json('config_sparql.json')
ENDPOINT = login['endpoint'][0]
USERNAME = login['username'][0]
PASSWORD = login['password'][0]

# create sparql object
sparql = practicalSPARQL.practicalWrapper(ENDPOINT)
sparql.setCredentials(USERNAME, PASSWORD)

print("--- Querying ENDPOINT: {} ---".format(ENDPOINT))

--- Querying ENDPOINT: http://devmeta.sphaera.mpiwg-berlin.mpg.de/sparql ---


In [3]:
q = practicalSPARQL.stringify_SPARQL('elements_query_w_iiif_230724.sparql')    # select data from the ttl file as a dataframe
df = sparql.select_as_dataframe(q) 
len(df)

21115

In [4]:
df = df.drop_duplicates(subset=['cluster_name'], keep='first')
df

,images,cluster_name,cks,book,bid,part_or_adaption,part_or_adaption_label,custom_identifier,place,year,bb,pagenum,IIIFimage
0,http://dev.sphaera.mpiwg-berlin.mpg.de/contain...,SAC_SIL_01420,"CK_Right and Oblique Sphere, CK_Horizon",http://sphaera.mpiwg-berlin.mpg.de/id/item/21e...,1902,http://sphaera.mpiwg-berlin.mpg.de/id/part/47c...,Compendium by P. Valeriano,251,Paris,1552,"xywh=720,1179,492,485",85,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...
1,http://dev.sphaera.mpiwg-berlin.mpg.de/contain...,SAC_SIL_01528,"CK_Horizon, CK_Right and Oblique Sphere",http://sphaera.mpiwg-berlin.mpg.de/id/item/21e...,1902,http://sphaera.mpiwg-berlin.mpg.de/id/part/47c...,Compendium by P. Valeriano,251,Paris,1552,"xywh=1595,477,525,502",85,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...
2,http://dev.sphaera.mpiwg-berlin.mpg.de/contain...,SAC_SIL_03692,"CK_Horizon, CK_Right and Oblique Sphere",http://sphaera.mpiwg-berlin.mpg.de/id/item/21e...,1902,http://sphaera.mpiwg-berlin.mpg.de/id/part/47c...,Compendium by P. Valeriano,251,Paris,1552,"xywh=667,651,538,492",85,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...
3,http://dev.sphaera.mpiwg-berlin.mpg.de/contain...,SAC_SIL_00261,CK_Lunar Eclipse,http://sphaera.mpiwg-berlin.mpg.de/id/item/21e...,1902,http://sphaera.mpiwg-berlin.mpg.de/id/part/47c...,Compendium by P. Valeriano,251,Paris,1552,"xywh=1280,523,892,888",99,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...
4,http://dev.sphaera.mpiwg-berlin.mpg.de/contain...,SAC_SIL_00477,CK_Lunar Phases,http://sphaera.mpiwg-berlin.mpg.de/id/item/21e...,1902,http://sphaera.mpiwg-berlin.mpg.de/id/part/47c...,Compendium by P. Valeriano,251,Paris,1552,"xywh=477,153,702,1269",99,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20990,http://dev.sphaera.mpiwg-berlin.mpg.de/contain...,SAC_SIL_03460,CK_Superior Planets Lines and Motion,http://sphaera.mpiwg-berlin.mpg.de/id/item/75a...,2258,http://sphaera.mpiwg-berlin.mpg.de/id/adaption...,Commentary on Theorica by Peuerbach,349,Lyon,1581,"xywh=273,596,129,132",86,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...
20991,http://dev.sphaera.mpiwg-berlin.mpg.de/contain...,SAC_SIL_03495,CK_Superior Planets Lines and Motion,http://sphaera.mpiwg-berlin.mpg.de/id/item/75a...,2258,http://sphaera.mpiwg-berlin.mpg.de/id/adaption...,Commentary on Theorica by Peuerbach,349,Lyon,1581,"xywh=67,477,213,206",86,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...
21083,http://dev.sphaera.mpiwg-berlin.mpg.de/contain...,SAC_SIL_01903,CK_Centrality of Earth,http://sphaera.mpiwg-berlin.mpg.de/id/item/395...,2263,http://sphaera.mpiwg-berlin.mpg.de/id/part/078...,Strigel's treatise on the Sphere,779,Wittenberg,1565,"xywh=196,1770,1068,1148",46,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...
21085,http://dev.sphaera.mpiwg-berlin.mpg.de/contain...,SAC_SIL_03198,CK_Right and Oblique Sphere,http://sphaera.mpiwg-berlin.mpg.de/id/item/395...,2263,http://sphaera.mpiwg-berlin.mpg.de/id/part/078...,Strigel's treatise on the Sphere,779,Wittenberg,1565,"xywh=380,1119,1560,763",31,http://db.sphaera.mpiwg-berlin.mpg.de/Scaler/I...


In [6]:
#download images

for entry in df.iterrows():
    downloadlink = entry[1].IIIFimage
    downloadlink = downloadlink.split("full")
    downloadlink_merged = downloadlink[0] + (entry[1].bb).split("=")[1] + "/!300,300" + downloadlink[1] 
    img_data = requests.get(downloadlink_merged).content
    cks = entry[1].cks.split(", ")
    for ck in cks:
        if not os.path.exists("images_ck/" + ck + "/"):
            os.makedirs("images_ck/" + ck + "/")
        with open("images_ck/" + ck + "/"+ entry[1].cluster_name +".jpg", 'wb') as handler:
            handler.write(img_data)